# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [3]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

,reviews
0,"✅ Trip Verified | I had visa issues, and hen..."
1,✅ Trip Verified | Singapore to Heathrow with...
2,✅ Trip Verified | I recently travelled from ...
3,Not Verified | I paid for seats 80 A and B on...
4,"Not Verified | The flight wasn’t that bad, alt..."


In [4]:
df.to_csv("data/BA_reviews.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [5]:
df.shape

(1000, 1)

In [6]:
df["reviews"].value_counts()

reviews
✅ Trip Verified |   I had visa issues, and hence, was debarred from flying. The ground staff at Mumbai was excellent (a woman named Brinelle) was very helpful and understanding. I had to rebook my tickets, and while the original journey was consisting of 3 legs, BA forced me to rebook the entire ticket, because their system did not allow them to rebook just that one leg (even though there was a gap of 13 days between the two legs, not like they were connecting flights). The telephone operator was rude, took 45 minutes and kept saying she needed 3-4 more minutes to give me clarity. It was horrendous. The final price they offered me for rebooking 2 of the 3 legs was much more than the original fare for all 3 legs. She was torturous, BA was horrendous because the website does not allow you to do all this. A very poor experience, in all.                                                                                                                                                   

In [7]:
import nltk 

In [8]:
import re 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation



In [10]:
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\LHCP\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LHCP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LHCP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
def clean_text(text):
    text = re.sub(r'[^\w\s]','',text) #remove puntuation marks
    text = text.lower() #convert to lowercase
    text = re.sub(r'\d+','',text) # remove numbers
    text = re.sub(r'\n',' ',text) # replace newlines with spaces
    return text

df['cleaned_text'] = df['reviews'].apply(clean_text)

**Removing stopwords**

In [12]:
stop_words = set(stopwords.words('english'))
df['cleaned_text'] = df['cleaned_text'].apply(lambda x:''.join([word for word in x.split() if
                                                                word not in stop_words
                                                                ]))

**Sentiment analysis**

In [14]:
sid = SentimentIntensityAnalyzer()
def get_sentiment(text):
    scores = sid.polarity_scores(text) # get sentiment score for text 
    # Classify as positive neutral or negative 
    return 'positive' if scores['compound'] > 0.05 else 'negative'if scores['compound'] < -0.05 else 'neutral'

df['sentiment'] = df['cleaned_text'].apply(get_sentiment)
print(df['sentiment'].value_counts())

sentiment
neutral    1000
Name: count, dtype: int64
